In [17]:
from pymongo import MongoClient
import requests
from colombia_config_impactu import endpoints
import pandas as pd
import re

In [20]:
pattern = r'^https?://[^/]+'

data={}
for i in endpoints.keys():
    if endpoints[i]["enabled"]:
        data[i]={"inst_name":endpoints[i]["name"], "base_url":endpoints[i]["url"],"home_page":re.match(pattern, endpoints[i]["url"]).group(0)}
        print(data[i]["home_page"])

https://bibliotecadigital.ccb.org.co
http://repositorio.unaula.edu.co:8080
https://repositorio.unal.edu.co
http://repositorio.pedagogica.edu.co
https://repositorio.uptc.edu.co
http://repositorio.unicauca.edu.co:8080
https://repositorio.utp.edu.co
https://repositorio.ucaldas.edu.co
https://repositorio.unicordoba.edu.co
https://repositorio.minciencias.gov.co
https://repository.unimilitar.edu.co
https://repositorio.unillanos.edu.co
https://repositorio.unicolmayor.edu.co
https://repositorio.unipacifico.edu.co
https://bibliotecadigital.udea.edu.co
http://repositorio.uac.edu.co
https://bibliotecadigital.univalle.edu.co
https://noesis.uis.edu.co
https://repositorio.unicartagena.edu.co
https://repository.ut.edu.co
https://bdigital.uniquindio.edu.co
https://repositorio.ufps.edu.co
https://repositorio.ucundinamarca.edu.co
https://repositorio.unisucre.edu.co
https://repositoryinst.uniguajira.edu.co
https://repository.udistrital.edu.co
https://repository.javeriana.edu.co
https://sie.car.gov.co
htt

In [21]:
len(data)

118

In [22]:
#data_valid

In [23]:
url_unpaywall="https://api.unpaywall.org/repository/endpoint/test/"

In [24]:
db=MongoClient()["oxomoc_colombia"]

In [25]:
not_found=[]
for i in data:
    rec=db[i+"_identity"].find_one()
    if rec:
        data[i]["repository_name"]=rec["repository_name"]
    else:
        not_found.append(i)
len(data)

118

In [15]:
#removed not found
for i in not_found:
    del data[i]

In [16]:
for i  in data:
    print(data[i]["home_page"])

https://bibliotecadigital.ccb.org
http://repositorio.unaula.edu
https://repositorio.unal.edu
http://repositorio.pedagogica.edu
https://repositorio.uptc.edu
http://repositorio.unicauca.edu
https://repositorio.utp.edu
https://repositorio.ucaldas.edu
https://repositorio.unicordoba.edu
https://repositorio.minciencias.gov
https://repository.unimilitar.edu
https://repositorio.unillanos.edu
https://repositorio.unicolmayor.edu
https://repositorio.unipacifico.edu
https://bibliotecadigital.udea.edu
http://repositorio.uac.edu
https://bibliotecadigital.univalle.edu
https://noesis.uis.edu
https://repositorio.unicartagena.edu
https://repository.ut.edu
https://bdigital.uniquindio.edu
https://repositorio.ufps.edu
https://repositorio.ucundinamarca.edu
https://repositorio.unisucre.edu
https://repositoryinst.uniguajira.edu
https://repository.udistrital.edu
https://repository.javeriana.edu
https://sie.car.gov
https://repository.usta.edu
https://bdigital.uexternado.edu
https://expeditiorepositorio.utadeo.e

In [27]:
for i in data:
    print(i)
    url=url_unpaywall+data[i]["base_url"]+"?verb=Identify"
    print(url)
    req=requests.get(url)
    if req.status_code == 200:
        data[i]["request_fail"]=False
        req_data=req.json()
        if req_data['results']['check0_identify_status']=='SUCCESS!' and req_data['results']['check1_query_status']=='SUCCESS!':
            data[i]["status_checks"]=True
        else:
            data[i]["status_checks"]=False
    else:
        data[i]["request_fail"]=True
        print("FAIL:"+url)

dspace_ccb
https://api.unpaywall.org/repository/endpoint/test/https://bibliotecadigital.ccb.org.co/server/oai/request?verb=Identify
dspace_unaula
https://api.unpaywall.org/repository/endpoint/test/http://repositorio.unaula.edu.co:8080/server/oai/request?verb=Identify
dspace_unal
https://api.unpaywall.org/repository/endpoint/test/https://repositorio.unal.edu.co/oai/request?verb=Identify
dspace_pedagogica
https://api.unpaywall.org/repository/endpoint/test/http://repositorio.pedagogica.edu.co/oai/request?verb=Identify
dspace_uptc
https://api.unpaywall.org/repository/endpoint/test/https://repositorio.uptc.edu.co/server/oai/request?verb=Identify
dspace_unicauca
https://api.unpaywall.org/repository/endpoint/test/http://repositorio.unicauca.edu.co:8080/oai/request?verb=Identify
dspace_utp
https://api.unpaywall.org/repository/endpoint/test/https://repositorio.utp.edu.co/server/oai/request?verb=Identify
dspace_ucaldas
https://api.unpaywall.org/repository/endpoint/test/https://repositorio.ucalda

In [32]:
data_out=[]

for i in data:
    if data[i]['request_fail'] == False:
        if data[i]['status_checks']:
            data_out.append([data[i]["inst_name"],data[i]["base_url"],data[i]["home_page"],data[i]["repository_name"]])


In [33]:
len(data_out)

45

In [34]:
df=pd.DataFrame(data_out,columns=["inst_name","base_url","home_page","repository_name"])

In [35]:
df

,inst_name,base_url,home_page,repository_name
0,FUNDACION UNIVERSITARIA EMPRESARIAL DE LA CAMA...,https://bibliotecadigital.ccb.org.co/server/oa...,https://bibliotecadigital.ccb.org.co,Biblioteca Digital
1,Universidad Autónoma Latinoamericana,http://repositorio.unaula.edu.co:8080/server/o...,http://repositorio.unaula.edu.co:8080,Universidad Autonoma Latinoamericana - UNAULA
2,UNIVERSIDAD PEDAGOGICA NACIONAL,http://repositorio.pedagogica.edu.co/oai/request,http://repositorio.pedagogica.edu.co,Repositorio Institucional Universidad Pedagógi...
3,UNIVERSIDAD PEDAGOGICA Y TECNOLOGICA DE COLOMB...,https://repositorio.uptc.edu.co/server/oai/req...,https://repositorio.uptc.edu.co,Repositorio Institucional UPTC
4,UNIVERSIDAD TECNOLOGICA DE PEREIRA - UTP,https://repositorio.utp.edu.co/server/oai/request,https://repositorio.utp.edu.co,Repositorio de la Universidad Tecnológica de P...
5,UNIVERSIDAD DE CALDAS,https://repositorio.ucaldas.edu.co/server/oai/...,https://repositorio.ucaldas.edu.co,Repositorio Universidad de Caldas
6,UNIVERSIDAD DE CORDOBA,https://repositorio.unicordoba.edu.co/server/o...,https://repositorio.unicordoba.edu.co,Repositorio Universidad de Córdoba
7,UNIVERSIDAD DEL ATLANTICO,http://repositorio.uac.edu.co/oai/request,http://repositorio.uac.edu.co,Repositorio digital de la Universidad Autónoma...
8,UNIVERSIDAD DEL VALLE,https://bibliotecadigital.univalle.edu.co/serv...,https://bibliotecadigital.univalle.edu.co,Repositorio Institucional Universidad del Valle
9,UNIVERSIDAD DE CARTAGENA,https://repositorio.unicartagena.edu.co/server...,https://repositorio.unicartagena.edu.co,Biblioteca Digital Universidad de Cartagena


In [36]:
df.to_csv("dspace_openalex.csv")